In [0]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('drive/My Drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
from __future__ import print_function
from keras.callbacks import LambdaCallback
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.layers import LSTM
from keras.layers import Embedding
from collections import Counter
from keras.preprocessing.sequence import pad_sequences

import numpy as np
import random
import sys
import io
import string

In [0]:
# Read data and check length
text = open('marvel_data.txt', encoding="utf8").read().lower()
print('text length', len(text))

text length 28213380


In [0]:
# Reduce (stop memory crashes - may be worth looking into memory reduction techniques instead)
reduced_text = text[:1000000]

In [0]:
# Random line
print(text[20086:20480])

 she then tied him up until the police arrived on the scene and showed them images of chord and the dealings of the ten rings which led to his arrest. midnight's fire has very subtle abilities, due to the energies of the well of all things. his physical attributes are at the peak of human possibility. he has enhanced speed, strength, agility, and sensory perception on par with black panther.


In [0]:
# Required cleans
def clean_doc(doc):
	# replace '--' with a space ' '
	doc = doc.replace('--', ' ')
	# split into tokens by white space
	tokens = doc.split()
	# remove punctuation from each token
	table = str.maketrans('', '', string.punctuation)
	tokens = [w.translate(table) for w in tokens]
	# remove remaining tokens that are not alphabetic
	tokens = [word for word in tokens if word.isalpha()]
	# make lower case
	tokens = [word.lower() for word in tokens]
	return tokens

	
tokens = clean_doc(reduced_text)

In [0]:
# Example of tokens
print(tokens[:200])
print('Total Tokens: %d' % len(tokens))
print('Unique Tokens: %d' % len(set(tokens)))

['unnamed', 'cousin', 'due', 'to', 'false', 'accusation', 'spread', 'by', 'afari', 'against', 'him', 'and', 'the', 'black', 'spears', 'commander', 'amboola', 'they', 'were', 'both', 'suspected', 'by', 'queen', 'tananda', 'of', 'plotting', 'against', 'her', 'and', 'were', 'arrested', 'amboola', 'was', 'killed', 'by', 'a', 'demon', 'released', 'by', 'the', 'sorcerer', 'muru', 'agent', 'of', 'the', 'actual', 'plotter', 'tuthmes', 'but', 'aahmes', 'fate', 'remained', 'unknown', 'aaidan', 'blomfield', 'was', 'the', 'third', 'man', 'to', 'call', 'himself', 'the', 'unicorn', 'he', 'was', 'an', 'agent', 'of', 'stockpile', 'attempting', 'to', 'raid', 'stark', 'enterprises', 'for', 'morgan', 'stark', 'after', 'the', 'apparent', 'death', 'of', 'tony', 'stark', 'he', 'claimed', 'to', 'be', 'an', 'old', 'foe', 'of', 'iron', 'mans', 'but', 'it', 'is', 'unknown', 'if', 'he', 'really', 'was', 'or', 'was', 'just', 'riding', 'on', 'the', 'reputation', 'of', 'the', 'original', 'unicorn', 'he', 'displayed

In [0]:
# Count the tokens
Counter(tokens).most_common(10)

[('the', 11274),
 ('to', 6773),
 ('and', 5352),
 ('of', 4666),
 ('a', 3580),
 ('was', 3255),
 ('his', 2787),
 ('in', 2429),
 ('he', 2349),
 ('by', 1586)]

In [0]:
# Organise into sequences
length = 41
sequences = list()
for i in range(length, len(tokens)):
	# select sequence of tokens
	seq = tokens[i-length:i]
	# convert into a line
	line = ' '.join(seq)
	# store
	sequences.append(line)
print('Total Sequences: %d' % len(sequences))

Total Sequences: 167559


In [0]:
# Encode the sequences
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sequences)
sequences = tokenizer.texts_to_sequences(sequences)

In [0]:
# Check the size of the vocab
vocab_size = len(tokenizer.word_index) + 1
print(vocab_size)

In [0]:
# Split into the input text and the output text
sequences = np.array(sequences)
X, y = sequences[:,:-1], sequences[:,-1]
y = to_categorical(y, num_classes=vocab_size)
seq_length = X.shape[1]

In [0]:
# Create an RNN model
model = Sequential()
model.add(Embedding(vocab_size, 50, input_length=seq_length))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(100))
model.add(Dense(100, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())




Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 40, 50)            755800    
_________________________________________________________________
lstm_1 (LSTM)                (None, 40, 100)           60400     
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_1 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_2 (Dense)              (None, 15116)             1526716   
Total params: 2,433,416
Trainable params: 2,433,416
Non-trainable params: 0
_________________________________________________________________
None


In [0]:
# Compile & Fit
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, batch_size=128, epochs=100)



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Epoch 1/100





167559/167559 [==============================] - 215s 1ms/step - loss: 7.1368 - acc: 0.0711
Epoch 2/100
167559/167559 [==============================] - 206s 1ms/step - loss: 6.6924 - acc: 0.0886
Epoch 3/100
167559/167559 [==============================] - 207s 1ms/step - loss: 6.4756 - acc: 0.1114
Epoch 4/100
167559/167559 [==============================] - 206s 1ms/step - loss: 6.2979 - acc: 0.1272
Epoch 5/100
167559/167559 [==============================] - 206s 1ms/step - loss: 6.1424 - acc: 0.1353
Epoch 6/100
167559/167559 [==============================] - 205s 1ms/step - loss: 6.0633 - acc: 0.1382
Epoch 7/100
167559/167559 [==============================] - 203s 1ms/step - loss: 5.9131 - acc: 0.1457
Epoch 8/100
167559/167559 [==============================] - 204s 1ms/step - loss: 5.7953 - acc: 0.1525
Epoch 9/100
167559/167559 [==============================] - 204

In [0]:
# Save model output
from pickle import dump
model.save('marvel_model1.h5')
dump(tokenizer, open('tokenizer.pkl', 'wb'))

In [0]:
# Ask Martyn to write me a 40 word opening sentence
seed_text = 'Born in 1993, bullied in school for having small feet Martyn grew up on a local wheat farm. Lifting bails made him a strong man, far stronger than the average human. One day during a solar eclipse something odd happened'
encoded = tokenizer.texts_to_sequences([seed_text])[0]
encoded_array = np.array([encoded])
encoded_array.shape

In [0]:
# Predict the output
predicted = model.predict_classes(encoded_array, verbose=0)

ValueError: ignored

In [0]:
# Generate a sequence
def generate_seq(model, tokenizer, seq_length, seed_text, n_words):
	result = list()
	in_text = seed_text
	# generate a fixed number of words
	for _ in range(n_words):
		# encode the text as integer
		encoded = tokenizer.texts_to_sequences([in_text])[0]
		# truncate sequences to a fixed length
		encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
		# predict probabilities for each word
		yhat = model.predict_classes(encoded, verbose=0)
		# map predicted word index to word
		out_word = ''
		for word, index in tokenizer.word_index.items():
			if index == yhat:
				out_word = word
				break
		# append to input
		in_text += ' ' + out_word
		result.append(out_word)
	return ' '.join(result)


In [0]:
generated = generate_seq(model, tokenizer, seq_length, seed_text, 50)
print(generated)

to the rigellians and nick fury will triggered his colorblindness and exchanging their japanese timeline grumlin was to combat fields hurani is a urgent detector radiate the wicked arno stark were being controlled by terror in the surface cap fell together facing shiv as part owner anubis fought for leaking


In [0]:
# Full sentence, with a few improvements (capital letters / commas)

"[HUMAN MADE:] Born in 1993, bullied in school for having small feet Martyn grew up on a local wheat farm. Lifting bails made him a strong man, far stronger than the average human. \n
One day during a solar eclipse something odd happened [MODEL MADE:] to the Rigellians and Nick Fury's will triggered his colorblindness and exchanging their Japanese timeline, \n
Grumlin was to combat fields. Hurani is an urgent detector, radiate the wicked Arno Stark were being controlled by terror in the surface. Cap fell together facing Shiv as part owner Anubis fought for leaking"

In [0]:
# We're not there yet!